G= (T, N, R,S)

Conjunto de terminales: T= {time, flies, arrow, an, like}

Conjunto de no terminales: N= {S, NP, VP, PP, Det, Nominal, Verb, Preposition, Noun}

Conjunto de reglas:

 R={

 S → NP VP				0.800

 NP → time | flies | arrow		0.002

 NP → Det Nominal			0.300

 NP → Nominal Nominal		0.200

 Nominal → time | flies | arrow	0.002

 Nominal → Nominal Noun		0.100

 Nominal → Nominal PP		0.200

 VP → time				0.004

 VP → flies| like			0.008

 VP → Verb NP				0.300

 VP → Verb PP				0.200

 PP → Preposition NP			0.100

 Verb → time				0.010

 Verb → flies| like			0.020

 Noun → time | flies | arrow		0.010

 Det → an				0.050

 Preposition → like			0.050

 }
 

La variable `file_grammar` que aparece a continuación emula las reglas gramaticales escritas en un archivo de texto. En este caso se han escrito las reglas propias del ejercicio.

In [1]:
file_grammar = """
S -> NP VP
NP -> Det Nominal
NP -> Nominal Nominal
Nominal -> Nominal Noun
Nominal -> Nominal PP
VP -> Verb NP
VP -> Verb PP
PP -> Preposition NP
"""

Se crea el lexicón, de forma común, en lingüística se dice que un elemento del lexicón en la entrada de un diccionario, entonces se define como tal en el código.

In [2]:
lexicon_dictionary = {
    'NP': {'time', 'flies', 'arrow'},
    'Nominal': {'time', 'flies', 'arrow'},
    'VP': {'time', 'flies', 'like'},
    'Verb': {'time', 'flies', 'like'},
    'Noun': {'time', 'flies', 'arrow'},
    'Det': {'an'},
    'Preposition': {'like'}
}

Se expresan las reglas gramaticales y lexicas en forma de tuplas como claves de una probabilidad que se usará para obtener las probabilidades de cada árbol obtenido por el algoritmo. Se eligen tuplas porque son fáciles de manejar, similar a los lenguajes Lisp.

In [3]:
probabilities = {
    ('Det', 'an'): 0.05,
    ('NP', ('Det', 'Nominal')): 0.3,
    ('NP', ('Nominal', 'Nominal')): 0.2,
    ('NP', 'arrow'): 0.002,
    ('NP', 'flies'): 0.002,
    ('NP', 'time'): 0.002,
    ('Nominal', ('Nominal', 'Noun')): 0.1,
    ('Nominal', 'arrow'): 0.002,
    ('Nominal', 'flies'): 0.002,
    ('Nominal', 'time'): 0.002,
    ('Nominal', ('Nominal', 'PP')): 0.2,
    ('Noun', 'arrow'): 0.01,
    ('Noun', 'flies'): 0.01,
    ('Noun', 'time'): 0.01,
    ('PP', ('Preposition', 'NP')): 0.1,
    ('Preposition', 'like'): 0.05,
    ('S', ('NP', 'VP')): 0.8,
    ('VP', ('Verb', 'PP')): 0.2,
    ('VP', 'flies'): 0.008,
    ('VP', 'like'): 0.008,
    ('VP', ('Verb', 'NP')): 0.3,
    ('VP', 'time'): 0.004,
    ('Verb', 'flies'): 0.02,
    ('Verb', 'like'): 0.02,
    ('Verb', 'time'): 0.01
    }

La función `grammar_rules() recibe como argumento un texto con reglas gramaticales y las convierte en un diccionario que permite buscar a partir de los nodos hijos de un árbol un posible padre que complete la estructura, esta forma es básica para facilitar la búsqueda en el algoritmo CKY.

In [4]:
def grammar_rules(grammar_buffer):
    '''
    This function parse grammar rules writen in a text variable or a file.

    The text file contain grammar rules writen like:

    NP -> NP VP

    This function doesn't parse lexical rules.

    This function return a dictionary with parents indexed by children, like this:

    {('Det', 'Nominal'): ['NP'],
     ('NP', 'VP'): ['S'],
     ('Verb', 'PP'): ['VP']}

        Parameters:
            grammar_buffer (str): Text with grammar rules
        
        Returns:
            find_parent (dict): Dictionary with the rules indexed by children
    '''
    rules = []
    # leer cada línea del texto
    for line in grammar_buffer.strip().split("\n"):
        # separar cada línea en sus dos miembros
        parent, chidren = line.strip().split("->")
        # Quitar espacio en blanco posterior
        parent = parent.strip()
        # Distinguir los hijos
        chidren = chidren.split()
        #crear la regla (parent, (c1, c2)) como en lisp ;)
        rule = (parent, tuple(chidren))
        rules.append(rule)
    # Crear un diccionario con las reglas invertidas, eso permite buscar al nodo padre
    find_parent = dict()
    for parent, (lc, rc) in rules:
        # Si no existe la entrada, se crea
        if (lc, rc) not in find_parent:
            find_parent[lc, rc] = []
        # Se agrega el caso
        find_parent[lc,rc].append(parent)
    return find_parent

#parents = set(val[0] for val in list(find_parent.values()))
#for i in list(lexicon.keys()):
#    parents.add(i)


Ahora se implementa el algoritmo CKY, mismo que recibe como entrada una oración en forma de cadena de texto, un diccionario con reglas gramaticales y un lexicón adecuado a la gramática. Opcionalmente, se puede definir el argumento de entrada `verbose´ al valor `True`, de esta manera se mostrará en pantalla cada uno de los pasos que sigue el algoritmo para crear la matriz correspondiente. Esta matriz también tendrá forma de diccionario.

In [5]:
def cky_parser(sentence, grammar, lexicon, verbose=False):
    ''' 
    The function cky_parser implements CKY algorithm.

        Parameters:
            sentence (str): A sentence to be analyzed
            grammar (dic): Dictionary with the grammar rules indexed by children
            lexicon (dic): Dictionary with lexical rules indexed by PoS
            verbose (bool): If it is True, show every step in the algorithm
        Returns:
            matrix_cky (dic): Dictionary with all possible cells written by the algorithm
    '''
    words = sentence.lower().split()
    if verbose:
        print(words)
    matrix_cky = {}
    for i in range(len(words)):
        # Las ordenadas van del 0 al 4
        ordinate = 0
        if verbose:
            print("0: i=",i)
        # Las abscisas van de 1 a 5
        for abscissa in range(i+1, len(words)+1):
            # Se agrega una celda
            matrix_cky[(ordinate, abscissa)] = []
            if verbose:
                print("1: abscissa=",abscissa,"ordinate=",ordinate,"a-o=",abscissa-ordinate)
            # Si se opera sobre la diagonal
            if(abscissa-ordinate==1):
                # Combina la palabra con las categorías del lexicón
                for key in lexicon_dictionary:
                    if verbose:
                        print("2: key=",key,"word=",words[ordinate])
                    # Si la combinación existe en el lexicón se agrega a la matriz
                    if(words[ordinate] in lexicon[key]):
                        matrix_cky[(ordinate,abscissa)].append((key,0,words[ordinate],words[ordinate]))
                        if verbose:
                            print("2:",matrix_cky)
            # Si hay que operar sobre dos celdas
            elif(abscissa-ordinate>1):
                if verbose:
                    print("3:","(",ordinate,",",abscissa,")")
                # Obtener los valores de las celdas
                for index in range(abscissa-ordinate-1):
                    left = matrix_cky[(ordinate,abscissa-1-index)]
                    down = matrix_cky[(abscissa-1-index,abscissa)]
                    if verbose:
                        print("4: index=",index)
                        print("4: left=",left,"\n  ","down=",down)
                    # Si una de las celdas esta vacía no se hace nada
                    if not left or not down:
                        if verbose:
                            print("5: Nothing to do")
                    else:
                        # Combinar los valores de las celdas
                        for a in left:
                            for b in down:
                                # Obtiene el primer elemento de la tupla
                                # Crea la tupla para buscarla en las reglas
                                if((a[0],b[0]) in grammar):
                                    matrix_cky[(ordinate,abscissa)].append(
                                        (grammar[(a[0],b[0])][0],
                                        abscissa-1-index,a[0],b[0])
                                        )
                                    if verbose:
                                        print("6: add=",grammar[(a[0],b[0])][0],abscissa-1-index,a[0],b[0])
                                else:
                                    if verbose:
                                        print("6: Nothing to do")
            if verbose:
                print(matrix_cky)
                print("9: Next Ordinate")
            ordinate+=1
    return matrix_cky

In [6]:
def tree (matrix,cell,parent="S",index=0):
    '''
    Create a tree from a matrix created by the CKY algorithm.
    See def cky_parser(sentence, grammar, lexicon, verbose=False) function.

        Parameters:
            matrix (dic): Dictionary with CKY algorithm information
            cell (tuple): Node in the matrix to start the analysis
            parent (str): Axiom in the grammar or parent node value for the tree
            index (int): If there is more than one value in cell, this variable let choose which one analyze
        
        Returns:
            list (list): A list with trees in list shape
    '''
    list = []
    # Revisa que la lista no este vacía
    if len(matrix[cell]) == 0:
        return list
    # Revisa que el índice sea 0, significa que es una hoja
    if matrix[cell][0][1] == 0:
        # Encuentra la hoja que corresponde
        if matrix[cell][index][0] == parent:
            return [matrix[cell][index][0], matrix[cell][index][2]]
        else:
            # Busca en otra hoja
            if index+1 < len(matrix[cell]):
                return tree(matrix,cell,parent,index+1)
            else:
                return list
    # Si no es una hoja, revisa si es la opción correcta
    elif (matrix[cell][index][0]) == parent:
        # Agrega el símbolo
        list.append(matrix[cell][index][0])
        # Buscar hijos
        child = []
        # Hijo derecho
        child.append(
            tree(matrix,(cell[0],matrix[cell][index][1]),matrix[cell][index][2]))
        # Hijo izquierdo
        child.append(
            tree(matrix,(matrix[cell][index][1],cell[1]),matrix[cell][index][3]))
        #Agrega child a la lista
        list.append(child)
        return list
    else:
        if index+1 < len(matrix[cell]):
            return tree(matrix,cell,parent,index+1)
        else:
            return list

In [7]:
def find_solutions(sentence,grammar,lexicon,axiom,verbose=False):
    '''
    Find all possible solutions of a CKY analysis in a sentence. 
    '''
    matrix_cky = cky_parser(sentence,grammar,lexicon,verbose)
    N = len(sentence.split())
    solutions = []
    for i in enumerate(matrix_cky[(0,N)]):
        if i[1][0] == axiom:
            solutions.append(tree(matrix_cky,(0,N),index=i[0]))
    return solutions

In [8]:
from pprint import pprint
result = find_solutions("Time flies like an arrow",
                        grammar_rules(file_grammar),
                        lexicon_dictionary,
                        "S")
pprint(result)

[['S',
  [['NP', [['Nominal', 'time'], ['Nominal', 'flies']]],
   ['VP', [['Verb', 'like'], ['NP', [['Det', 'an'], ['Nominal', 'arrow']]]]]]],
 ['S',
  [['NP', 'time'],
   ['VP',
    [['Verb', 'flies'],
     ['PP',
      [['Preposition', 'like'],
       ['NP', [['Det', 'an'], ['Nominal', 'arrow']]]]]]]]]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=16bbac0e-c0cf-475a-9977-7067c93e2eaf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>